In [ ]:
import torch
from torch import nn
from torchvision import datasets, transforms
import torch.functional as F
from tqdm import tqdm


# Linear layer network (fully-connected, dense layer)

torch.nn.module을 사용하여 linear model initialization을 할 때, 아래와 같은 argument를 정해줘야 합니다.

- in_features (necessary): input vector dimension을 나타내는 `integer` number
- out_features (necessary): output vector dimension을 나타내는 `integer` number
- bias: bias parameter를 사용할지 말지에 대한 `bool` type number
- device: training, inference에 어떤 computational resource를 사용할지에 대한 argument. 보통 `cpu` or `cuda` 를 사용
- dtype: 모델 파라미터에 사용할 number의 data type으로, 보통 `float32`

In [ ]:
input_size = 784
output_size = 10

linear = nn.Linear(
    in_features=input_size, 
    out_features=output_size,
    bias=True,
    device='cpu',
    dtype=torch.float32
    )

linear network가 어떤 hyperparameter로 initialization 되었는지 확인합니다.

In [ ]:
linear

Linear(in_features=784, out_features=10, bias=True)

weight, bias의 tensor size를 확인합니다.

In [ ]:
params = list(linear.parameters())
print(
    f"size of weight parameters: {params[0].size()}\n",
    f"size of bias parameters: {params[1].size()}",
    )

size of weight parameters: torch.Size([10, 784])
 size of bias parameters: torch.Size([10])


tensor of random numbers를 사용해서, initialization된 linear network를 어떻게 쓸 수 있는지 확인해보겠습니다.

먼저, 아래와 같은 debugging용 데이터를 생성합니다.

- X: random tensor of (batch_size, input_size)

- Y: arbitrary classification label fixed at 0.

Cross-entropy loss 와 stochastic gradient descent optimizer 를 사용하겠습니다.

In [ ]:
batch_size = 12

# define example input and label
input = torch.randn(batch_size, input_size, requires_grad=True)
label = torch.zeros(batch_size, dtype=torch.long)

# define loss function for classification problem
loss_function = nn.CrossEntropyLoss()

# define optimizer to update parameters given gradients
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

tensor 연산을 할 때, 모든 tensor는 동일한 device에 올라가 있어야 합니다.

model parameters와 the input tensor 또한 tensor 연산에 사용될 것이기 때문에 같은 device로 옮겨줄 필요가 있습니다.

예를 들어, 만약 model parameters는 `cpu`에, input tensor는 `cuda`에 올라가 있다면 에러가 나게 됩니다.

이 튜토리얼에서는 `cpu`로 device를 통일하겠습니다.

In [ ]:
# check whether the input and model parameters are on the same device
print(f"input device: {input.device}\n", f"model device: {linear.weight.device}")

input device: cpu
 model device: cpu


## Forward propagation

In [ ]:
# call the forward function of the instance, linear
out = linear(input)

In [ ]:
# calculate cross entropy loss between the probability distribution out and the label
loss = loss_function(out, label)

# check the loss value
loss

tensor(2.4582, grad_fn=<NllLossBackward0>)

# Automatic backpropagation using autograd

딥러닝 모델의 파라미터 업데이트를 위해서는 backpropagation 알고리즘이 구현되어 있어야 하는데요.

torch의 `autograd` library를 사용함으로써, backpropagation을 구현하지 않고도 모델 업데이트를 편리하게 할 수 있습니다.

`torch.nn` module은 단순히 `forward` 과정에서 모델 파라미터들이 loss function 계산에 참여하는 과정을 자동으로 추적한 뒤, backpropagation이 가능하도록 미분해줍니다.


먼저, backpropagation을 하기 전에, 혹시나 남아있을 수 있는 gradient를 `zero_grad()`를 사용하여 초기화한 뒤, 남아 있는 gradient가 있는지 확인합니다.

In [ ]:
# before do backpropagation, we need to zero the gradient
linear.zero_grad()

# check the gradient before backpropagation
linear.weight.grad

이제, backward() method를 호출하여 gradient 정보를 업데이트해보겠습니다.

In [ ]:
# do backpropagation
loss.backward()

# check the gradient after backpropagation
linear.weight.grad

tensor([[ 0.4533, -0.4148,  0.2939,  ..., -0.1157,  0.0359,  0.2141],
        [-0.0746,  0.0418, -0.0315,  ...,  0.0158, -0.0022, -0.0087],
        [-0.0465,  0.0325, -0.0288,  ...,  0.0125, -0.0085,  0.0018],
        ...,
        [-0.0494,  0.0545, -0.0378,  ...,  0.0194, -0.0065,  0.0075],
        [-0.0352,  0.0202, -0.0250,  ...,  0.0111, -0.0182, -0.0106],
        [-0.0218,  0.0653, -0.0575,  ...,  0.0380,  0.0257, -0.0833]])

`loss`를 계산하는 과정에서 추적한 gradient 정보를 사용해, `backward()` 함수를 호출함으로써 backpropagation을 실행합니다.

각 model parameter들은 `loss` 계산에서 기여한 만큼의 gradient를 저장하게 됩니다.

backpropagation 은 gradient 계산까지만을 의미하며, 이 시점에서는 아직 model parameter들은 업데이트되지 않았습니다.

추후의 비교를 위해, 업데이트 되기 전의 model parameter를 복사해둡니다.

In [ ]:
previous_linear_weight = linear.weight.clone()

linear model의 parameter를 `backward()`로 계산되었던 gradient 및 SGD optimizer를 사용하여 업데이트해보겠습니다.

방법은 아주 간단한데, `optimizer`의 `step()` method를 호출하는 것입니다.

In [ ]:

optimizer.step()

업데이트 이후,

- `previous_model.parameter`
- `updated_model.parameter`

가 각각 달라졌는지 확인해보겠습니다.

In [ ]:
# check equality of the tensors after update
print(torch.equal(linear.weight, previous_linear_weight))

False


# Multi-Layer Perceptron (MLP)

## Neural Network building using torch.nn module

`torch.nn` module을 사용해 deep neural network를 정의해줄 수 있습니다.

deep neural network는 parameter update에 gradient based optimization을 사용합니다.

이를 위해서는 gradient backpropagation algorithm이 구현되어 있어야하는데요.

`torch.nn` module을 사용할 경우, `autograd`를 통한 gradient 계산을 `torch` library가 자동으로 해주기 때문에 편리합니다.

아래는 그림은 Multi-Layer Perceptron의 구조에 대한 그림입니다.

![image](https://drive.google.com/uc?export=download&id=1YDra8zUcoZHUypmgPwfGKQtef7HFT8UP)

`torch.nn`을 사용해서 Multi-Layer Perceptron model을 정의해보겠습니다.

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x
    
mlp = MLP()
mlp


MLP(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=-1)
)

`class MLP(nn.Module):`

nn.Module class를 상속받은 MLP model을 선언합니다.

nn.Module을 상속받아야만 `autograd` 등 여러 유용한 기능을 사용할 수 있습니다.

`nn.Linear(784, 128)`

one layer linear model을 선언합니다.

이 경우, input feature는 784개, output feature는 128개가 됩니다.

`nn.ReLU()`, `nn.Softmax(dim=-1)`

activation function 입니다.

`def forward(self, x):`

nn.Module 을 상속받은 class라면 구현해야 하는 method입니다.

`python`의 `__call__()` method를 사용하여 호출되는데요.

`forward` method 내에서 이루어지는 계산은 `autograd에` 의해 automatic differentiation이 이루어져 gradient를 추적하게 됩니다.

위 예시에서는 각각의 linear layer를 hard coding하여 선언하였는데요.

layer depth가 깊어지고, 각각의 layer의 hyperparamter가 서로 다르면 손이 아플 수 있습니다.

예를 들어 linear layer를 10개 이상 쌓은 model을 이런 식으로 작성하고 싶지는 않을 것입니다.

이런 경우, model initialization을 편하게 할 수 있는 방법들을 사용하고 싶을 수 있습니다.

아래 예시는 `nn.ModuleList` 를 활용해 linear layer initialization의 hard coding을 자동화한 것입니다.

In [ ]:
class MLP2(nn.Module):
    def __init__(self, hidden_features: list):
        super(MLP2, self).__init__()
        mlp = []
        for i in range(len(hidden_features) -1 ):
            mlp.append(nn.Linear(hidden_features[i], hidden_features[i + 1]))
            mlp.append(nn.ReLU())
        mlp.append(nn.Softmax(dim=-1))

        self.mlp = nn.ModuleList(mlp)

    def forward(self, x):
        for layer in self.mlp:
            x = layer(x)
        
        return x
    
mlp2 = MLP2([784] + [100 for i in range(8)] + [10])

In [ ]:
mlp2

MLP2(
  (mlp): ModuleList(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): ReLU()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): ReLU()
    (8): Linear(in_features=100, out_features=100, bias=True)
    (9): ReLU()
    (10): Linear(in_features=100, out_features=100, bias=True)
    (11): ReLU()
    (12): Linear(in_features=100, out_features=100, bias=True)
    (13): ReLU()
    (14): Linear(in_features=100, out_features=100, bias=True)
    (15): ReLU()
    (16): Linear(in_features=100, out_features=10, bias=True)
    (17): ReLU()
    (18): Softmax(dim=-1)
  )
)

linear layer의 feature수가 표기된 list[int]를 argument로 사용하면, 

아래 예시는 `nn.Sequential` 를 활용해 linear layer initialization의 hard coding을 자동화한 것입니다.

In [ ]:
class MLP3(nn.Module):
    def __init__(self, hidden_features: list):
        super(MLP3, self).__init__()
        mlp = []
        for i in range(len(hidden_features) -1 ):
            mlp.append(nn.Linear(hidden_features[i], hidden_features[i + 1]))
            mlp.append(nn.ReLU())
        mlp.append(nn.Softmax(dim=-1))

        self.mlp = nn.Sequential(*mlp)

    def forward(self, x):
        x = self.mlp(x)
        
        return x
    
mlp3 = MLP3([784] + [100 for i in range(8)] + [10])

In [ ]:
mlp3

MLP3(
  (mlp): Sequential(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): ReLU()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): ReLU()
    (8): Linear(in_features=100, out_features=100, bias=True)
    (9): ReLU()
    (10): Linear(in_features=100, out_features=100, bias=True)
    (11): ReLU()
    (12): Linear(in_features=100, out_features=100, bias=True)
    (13): ReLU()
    (14): Linear(in_features=100, out_features=100, bias=True)
    (15): ReLU()
    (16): Linear(in_features=100, out_features=10, bias=True)
    (17): ReLU()
    (18): Softmax(dim=-1)
  )
)

# Convolutional Neural Network

2D convolution과 max pooling operator로 구성된, CNN classifier을 정의하겠습니다.

구조는 아래와 유사한데요.

![image](https://drive.google.com/uc?export=download&id=1DKuuGYDMmV8q03EvWCamXAhSjRfgkIkB)

2D convolution 을 정의할 때는 아래와 같은 정보가 필요합니다.

- `Number of input channel`: 입력 데이터의 채널 개수입니다. 입력 데이터가 흑백 이미지인 경우, 채널은 하나입니다(grayscale). RGB로 표현된 컬러 이미지의 경우, 빨강(Red), 초록(Green), 파랑(Blue)의 각 색상 구성요소를 위해 3개의 채널이 있습니다.
- `Number of output channel`: 출력 데이터의 채널 개수입니다.
- `Size of kernel function`: 입력 데이터와 합성곱(convolution)을 수행하는 필터의 크기입니다. 3의 커널 크기는 한 번에 입력의 3x3 픽셀 영역을 조사하는 3x3 필터를 의미합니다.

Max pooling을 정의할 때는 아래와 같은 정보가 필요합니다.

- `Size of kernel function`: 위와 동일
- `Size of stride`: 커널 함수가 적용되는 픽셀 수입니다. 더 큰 스트라이드는 작은 스트라이드보다 특성 크기를 더 빠르게 감소시킵니다.

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # input channel 1, output channel 6, kernel size 5
        self.conv1 = nn.Conv2d(1, 6, 5) 
        self.pool = nn.MaxPool2d(2, 2) 
        # input channel 6, output channel 16, kernel size 5
        self.conv2 = nn.Conv2d(6, 16, 5) 
        self.fc1 = nn.Linear(16 * 4 * 4, 120) # an image size is reduced to 4x4 at this stage
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4) # reshaping the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # no need to apply softmax, as it's included in the CrossEntropyLoss
        return x

cnn = CNN()


In [ ]:
cnn

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

# Recurrent Neural Network

RNN model의 구조는 대략 아래 그림과 같습니다.

![image](https://drive.google.com/uc?export=download&id=18nFqcyTje4rv-DlFdFCBOMceYqXTjjqm)

하나의 shared RNN Cell을 사용해, 이전 time step의 hidden state와 현재 time step의 input을 받으면 현재 time step의 output state가 계산되는데요.

RNN model을 정의하는 데에는 아래와 같은 정보가 필요합니다.

- `input_size`: Dimensionality of input feature
- `hidden_size`: Dimensionality of hidden feature, which is used between the input layer and output layer
- `num_layers`: The number of RNN layers

In [ ]:
class RNN(nn.Module):
    def __init__(self, num_tokens, input_size, hidden_size, num_layers, dropout=0.5):
        super(RNN, self).__init__()
        self.ntoken = num_tokens
        self.drop = nn.Dropout(dropout)
        self.embedding = nn.Embedding(num_tokens, input_size)
        self.rnn = nn.RNN(
            input_size, hidden_size, num_layers, dropout=dropout
            )
        self.head_layer = nn.Linear(hidden_size, num_tokens)

        self.nhid = hidden_size
        self.nlayers = num_layers


    def forward(self, input, hidden):
        emb = self.drop(self.embedding(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.head_layer(output)
        decoded = self.head_layer.view(-1, self.ntoken)
        return F.log_softmax(decoded, dim=1), hidden

In [ ]:
rnn = RNN(10, 768, 768, 12)

In [ ]:
rnn

RNN(
  (drop): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(10, 768)
  (rnn): RNN(768, 768, num_layers=12, dropout=0.5)
  (head_layer): Linear(in_features=768, out_features=10, bias=True)
)